在沙盒环境，对多特征进行降维的方法

In [1]:
import sys, os
import pandas as pd
import numpy as np
os.environ['ULTRON_DATA'] = 'minden'
os.environ['IGNORE_WARNINGS'] = '0'

sys.path.insert(0, os.path.abspath('../'))

In [2]:
from ultron.env import *
from ultron.factor.dimension import DimensionLDA
from ultron.factor.dimension import DimensionPCA
from ultron.factor.dimension import DimensionKeans
from ultron.factor.dimension import DimensionCorrcoef

/var/log/ultron/2022-11-20.log


In [3]:
indicator_data = pd.read_csv(os.path.join(g_project_data, 'indicator.csv'), index_col=0)
indicator_data['trade_date'] = pd.to_datetime(indicator_data['trade_date'])
indicator_data.head()

,code,trade_date,preClosePrice,openPrice,highestPrice,lowestPrice,closePrice,turnoverVol,turnoverValue,p_change,atr21,atr14,key,date_week
0,300002,2018-10-15,3.07,3.08,3.12,2.99,3.00,9880442,30219635.16,-0.023065,0.140328,0.140576,328,0
1,300002,2018-10-16,3.00,3.02,3.05,2.92,2.93,10977328,32893642.21,-0.023610,0.139836,0.139821,1079,1
2,300002,2018-10-17,2.93,2.98,3.05,2.96,3.03,12858515,38703529.14,0.033560,0.138891,0.138405,1830,2
3,300002,2018-10-18,3.03,3.02,3.16,2.97,3.08,18707797,57650966.23,0.016367,0.141325,0.142091,2581,3
4,300002,2018-10-19,3.08,3.03,3.22,3.03,3.19,18550310,58488703.69,0.035091,0.143643,0.145513,3332,4


In [4]:
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, 
                           random_state=7, n_classes=10)
samples_data = pd.DataFrame(X)
samples_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,2.354877,-1.696746,1.619388,-1.196689,-2.854223,-2.009984,16.561288,2.572576,9.937798,0.434150,6.082749,2.126893,1.701003,3.321610,13.020485,-3.050345,2.063467,-3.333904,2.451475,-1.234552
1,2.020428,-1.627348,-2.276974,2.244418,0.109845,2.131642,-2.583680,-4.240268,0.754107,-0.533652,0.700613,-2.279139,-1.909432,-0.700650,2.578135,0.963961,4.082339,-0.282747,-7.281665,-0.910703
2,-1.024007,1.012764,1.055058,-0.999481,-0.519337,0.199497,-0.357431,1.845920,3.889152,1.538555,-3.838436,1.973530,1.760786,-4.018298,-3.972360,-0.378769,2.519827,3.839240,-1.635306,3.960509
3,1.909641,-3.609265,-2.307864,0.737232,-0.046262,1.361876,0.620245,-2.851875,-0.333685,1.240630,2.353382,-1.523927,0.885453,0.251016,0.323264,0.247489,3.731732,-1.800513,-1.373245,1.090436
4,3.049515,-0.768281,-2.263043,1.293180,-3.197585,-2.700083,6.344461,-1.923060,1.609508,-3.153406,0.879794,3.285045,-0.154221,1.674090,9.070814,-5.693244,8.458329,-0.845611,-3.382476,-2.733542


In [5]:
indicator_features = [col for col in indicator_data.columns if col not in ['code','trade_date','key','date_week']]
samples_features = [col for col in samples_data.columns]

采用KEANS对数据进行聚类，通过聚类的结果对数据进行降维，其中对聚类结果提供三种校验方式
- 1. squares
- 2. calinski_harabas
- 3. silhoette

#### LDA 降维

indicator 数据 此处采用squares评估

In [6]:
kmeans = DimensionKeans(method='squares', features=indicator_features, thresh=0.9)
data = kmeans.run(factors_data=indicator_data)
lda = DimensionLDA(features=indicator_features)
lda.run(data, 9)

2022-11-20 09:26:53,877 - [kmeans.py:101] - ultron - INFO - initialize kmeans range:range(1, 10)


fit kmeans::88.89%

2022-11-20 09:27:23,776 - [kmeans.py:103] - ultron - INFO - squares evaluate kmean best k
2022-11-20 09:27:23,795 - [kmeans.py:109] - ultron - INFO - kmean best 9


fit LDA::100.0%s::100.0%

,lda_1_1
0,4.069225
1,4.043454
2,3.975828
3,3.800058
4,3.788627
...,...
59593,-21.528942
59594,-8.702006
59595,-15.166921
59596,-7.349763


sample 数据 此处采用harabasz评估

In [7]:
kmeans = DimensionKeans(method='harabasz', features=samples_features, thresh=0.9)
data = kmeans.run(factors_data=samples_data)
lda = DimensionLDA(features=samples_features)
lda.run(data, 19)

2022-11-20 09:27:44,869 - [kmeans.py:101] - ultron - INFO - initialize kmeans range:range(2, 20)


fit kmeans::94.44%

2022-11-20 09:28:03,253 - [kmeans.py:103] - ultron - INFO - harabasz evaluate kmean best k
2022-11-20 09:28:03,347 - [kmeans.py:109] - ultron - INFO - kmean best 19


fit LDA::100.0%sz::100.0%

,lda_8_1,lda_8_2,lda_8_3,lda_8_4,lda_8_5,lda_8_6,lda_8_7,lda_8_8
0,2.942935,-3.013031,-5.684942,-0.029904,0.684251,0.308779,0.484954,-0.407370
1,2.099099,1.057692,1.009078,-0.685050,1.195168,-0.409955,0.032452,-1.178393
2,-0.331300,0.633161,0.718091,1.062051,0.138670,-1.378638,1.143328,1.141431
3,0.525187,0.899438,-0.502119,-0.157295,0.871619,-0.333069,-0.722591,-0.406699
4,3.629645,0.513506,-3.294930,-0.145149,0.289154,0.463176,0.034120,-0.709659
...,...,...,...,...,...,...,...,...
995,1.308645,0.792882,-0.476530,-0.948583,0.886491,0.171245,0.001546,0.466114
996,0.667362,0.246409,-1.059309,-0.258601,0.379689,1.438103,-0.544770,1.197155
997,-0.246207,-1.553332,-1.924270,0.217399,-0.084425,0.189917,-0.099577,-0.725737
998,-0.286715,-2.602592,-1.466799,1.488931,0.971114,0.978107,-0.047917,0.007867


#### PCA 降维

samples 数据 此处采用squares评估

In [6]:
kmeans = DimensionKeans(method='squares', features=samples_features, thresh=0.9)
data = kmeans.run(factors_data=samples_data)
pca = DimensionPCA(features=samples_features)
pca.run(data, 9)

2022-11-20 08:57:39,245 - [kmeans.py:101] - ultron - INFO - initialize kmeans range:range(1, 20)


fit kmeans::94.74%

2022-11-20 08:58:09,497 - [kmeans.py:103] - ultron - INFO - squares evaluate kmean best k
2022-11-20 08:58:09,522 - [kmeans.py:109] - ultron - INFO - kmean best 19


fit PCA::100.0%s::100.0%

,pca_8_1,pca_8_2,pca_8_3,pca_8_4,pca_8_5,pca_8_6,pca_8_7,pca_8_8
0,5.200838,12.989139,18.713875,-0.006216,1.300128,4.209917,4.984662,3.138999
1,8.191675,-0.805795,-3.607105,-2.671120,3.734095,-0.611752,-0.696749,0.120433
2,0.908705,-4.143607,-1.790486,3.532638,0.528341,2.979978,-2.385537,3.999463
3,2.986832,-1.779030,1.469062,-0.659734,2.643855,-0.362076,2.225880,0.978208
4,12.090592,2.569083,10.916418,-1.292546,0.260783,1.254600,-0.528533,-1.621090
...,...,...,...,...,...,...,...,...
995,5.106296,-0.488623,1.021451,-3.590138,1.667203,1.291096,-2.498841,-3.430006
996,2.139585,0.719727,4.438423,-1.583901,2.498073,-0.449029,2.892128,-3.664176
997,-2.896016,5.346308,6.469143,0.842536,0.526262,-4.511725,-0.008302,0.948989
998,-4.576974,7.585456,5.524302,5.476117,2.475153,2.337029,3.054826,-0.026505


#### 相关性降维

In [7]:
from ultron.factor.dimension.corrcoef import FCorrType
from ultron.ump.similar.corrcoef import ECoreCorrType

##### 横截面降维

In [14]:
#### 皮尔逊
engine = DimensionCorrcoef(features=indicator_features, thresh=0.97, method=FCorrType.F_CS_CORR)
engine.run(factors_data=indicator_data,similar_type=ECoreCorrType.E_CORE_TYPE_PEARS)

2022-11-20 09:02:28,899 - [corrcoef.py:72] - ultron - INFO - preClosePrice,openPrice corr:0.999570497972484
2022-11-20 09:02:28,923 - [corrcoef.py:72] - ultron - INFO - preClosePrice,highestPrice corr:0.9990478497483362
2022-11-20 09:02:28,928 - [corrcoef.py:72] - ultron - INFO - preClosePrice,lowestPrice corr:0.9991053163172507
2022-11-20 09:02:28,933 - [corrcoef.py:72] - ultron - INFO - preClosePrice,closePrice corr:0.9987373125336174
2022-11-20 09:02:28,937 - [corrcoef.py:72] - ultron - INFO - preClosePrice,turnoverVol corr:-0.24062856803563468
2022-11-20 09:02:28,942 - [corrcoef.py:72] - ultron - INFO - preClosePrice,turnoverValue corr:0.4834280758794906
2022-11-20 09:02:28,946 - [corrcoef.py:72] - ultron - INFO - preClosePrice,p_change corr:-0.012692138802173
2022-11-20 09:02:28,957 - [corrcoef.py:72] - ultron - INFO - preClosePrice,atr21 corr:0.9447793467887488
2022-11-20 09:02:28,962 - [corrcoef.py:72] - ultron - INFO - preClosePrice,atr14 corr:0.936378096096153
2022-11-20 09:02

,code,trade_date,preClosePrice,turnoverVol,turnoverValue,p_change,atr21,key,date_week
0,300002,2018-10-15,3.07,9880442,3.021964e+07,-0.023065,0.140328,328,0
1,300002,2018-10-16,3.00,10977328,3.289364e+07,-0.023610,0.139836,1079,1
2,300002,2018-10-17,2.93,12858515,3.870353e+07,0.033560,0.138891,1830,2
3,300002,2018-10-18,3.03,18707797,5.765097e+07,0.016367,0.141325,2581,3
4,300002,2018-10-19,3.08,18550310,5.848870e+07,0.035091,0.143643,3332,4
...,...,...,...,...,...,...,...,...,...
59593,300724,2022-06-27,105.99,27152642,2.690173e+09,-0.116439,5.725449,678604,0
59594,300724,2022-06-28,94.34,14268590,1.357506e+09,0.025018,5.689475,679360,1
59595,300724,2022-06-29,96.73,22555360,2.032467e+09,-0.099143,5.893310,680116,2
59596,300724,2022-06-30,87.60,13768856,1.218505e+09,0.019780,5.816961,680872,3


In [15]:
#### 斯皮尔曼
engine = DimensionCorrcoef(features=indicator_features, thresh=0.97, method=FCorrType.F_CS_CORR)
engine.run(factors_data=indicator_data,similar_type=ECoreCorrType.E_CORE_TYPE_SPERM)

2022-11-20 09:02:58,712 - [corrcoef.py:72] - ultron - INFO - preClosePrice,openPrice corr:0.9984166498649267
2022-11-20 09:02:58,730 - [corrcoef.py:72] - ultron - INFO - preClosePrice,highestPrice corr:0.9982165161877965
2022-11-20 09:02:58,749 - [corrcoef.py:72] - ultron - INFO - preClosePrice,lowestPrice corr:0.9982770668239438
2022-11-20 09:02:58,768 - [corrcoef.py:72] - ultron - INFO - preClosePrice,closePrice corr:0.9995548176752853
2022-11-20 09:02:58,788 - [corrcoef.py:72] - ultron - INFO - preClosePrice,turnoverVol corr:-0.4405146369161698
2022-11-20 09:02:58,808 - [corrcoef.py:72] - ultron - INFO - preClosePrice,turnoverValue corr:0.5137074204155335
2022-11-20 09:02:58,829 - [corrcoef.py:72] - ultron - INFO - preClosePrice,p_change corr:-0.0030245699155284976
2022-11-20 09:02:58,848 - [corrcoef.py:72] - ultron - INFO - preClosePrice,atr21 corr:0.9696119425527334
2022-11-20 09:02:58,866 - [corrcoef.py:72] - ultron - INFO - preClosePrice,atr14 corr:0.9676419190255334
2022-11-20 

CS CORR::77.78%

2022-11-20 09:02:59,079 - [corrcoef.py:72] - ultron - INFO - p_change,turnoverVol corr:0.05453529164786261
2022-11-20 09:02:59,102 - [corrcoef.py:72] - ultron - INFO - p_change,turnoverValue corr:0.07416844460110916
2022-11-20 09:02:59,125 - [corrcoef.py:72] - ultron - INFO - p_change,atr21 corr:-0.0027489482868413573
2022-11-20 09:02:59,146 - [corrcoef.py:72] - ultron - INFO - p_change,atr14 corr:-0.0014242963108163063
2022-11-20 09:02:59,165 - [corrcoef.py:72] - ultron - INFO - atr21,turnoverVol corr:-0.3677299367888785
2022-11-20 09:02:59,185 - [corrcoef.py:72] - ultron - INFO - atr21,turnoverValue corr:0.5574813251539743
2022-11-20 09:02:59,205 - [corrcoef.py:72] - ultron - INFO - atr21,p_change corr:-0.002748948286841357
2022-11-20 09:02:59,224 - [corrcoef.py:72] - ultron - INFO - atr21,atr14 corr:0.9982633215159316


,code,trade_date,preClosePrice,turnoverVol,turnoverValue,p_change,atr21,key,date_week
0,300002,2018-10-15,3.07,9880442,3.021964e+07,-0.023065,0.140328,328,0
1,300002,2018-10-16,3.00,10977328,3.289364e+07,-0.023610,0.139836,1079,1
2,300002,2018-10-17,2.93,12858515,3.870353e+07,0.033560,0.138891,1830,2
3,300002,2018-10-18,3.03,18707797,5.765097e+07,0.016367,0.141325,2581,3
4,300002,2018-10-19,3.08,18550310,5.848870e+07,0.035091,0.143643,3332,4
...,...,...,...,...,...,...,...,...,...
59593,300724,2022-06-27,105.99,27152642,2.690173e+09,-0.116439,5.725449,678604,0
59594,300724,2022-06-28,94.34,14268590,1.357506e+09,0.025018,5.689475,679360,1
59595,300724,2022-06-29,96.73,22555360,2.032467e+09,-0.099143,5.893310,680116,2
59596,300724,2022-06-30,87.60,13768856,1.218505e+09,0.019780,5.816961,680872,3


In [16]:
#### 基于PEARS使用序列＋－符号
engine = DimensionCorrcoef(features=indicator_features, thresh=0.97, method=FCorrType.F_CS_CORR)
engine.run(factors_data=indicator_data,similar_type=ECoreCorrType.E_CORE_TYPE_SIGN)

2022-11-20 09:03:45,419 - [corrcoef.py:72] - ultron - INFO - preClosePrice,openPrice corr:0.9988086848551965
2022-11-20 09:03:45,423 - [corrcoef.py:72] - ultron - INFO - preClosePrice,highestPrice corr:0.9988086848551965
2022-11-20 09:03:45,427 - [corrcoef.py:72] - ultron - INFO - preClosePrice,lowestPrice corr:0.9988086848551965
2022-11-20 09:03:45,431 - [corrcoef.py:72] - ultron - INFO - preClosePrice,closePrice corr:1.0
2022-11-20 09:03:45,435 - [corrcoef.py:72] - ultron - INFO - preClosePrice,turnoverVol corr:0.9988086848551965
2022-11-20 09:03:45,439 - [corrcoef.py:72] - ultron - INFO - preClosePrice,turnoverValue corr:0.9988086848551965
2022-11-20 09:03:45,442 - [corrcoef.py:72] - ultron - INFO - preClosePrice,p_change corr:0.49293600456391157
2022-11-20 09:03:45,446 - [corrcoef.py:72] - ultron - INFO - preClosePrice,atr21 corr:1.0
2022-11-20 09:03:45,450 - [corrcoef.py:72] - ultron - INFO - preClosePrice,atr14 corr:1.0


,code,trade_date,preClosePrice,p_change,key,date_week
0,300002,2018-10-15,3.07,-0.023065,328,0
1,300002,2018-10-16,3.00,-0.023610,1079,1
2,300002,2018-10-17,2.93,0.033560,1830,2
3,300002,2018-10-18,3.03,0.016367,2581,3
4,300002,2018-10-19,3.08,0.035091,3332,4
...,...,...,...,...,...,...
59593,300724,2022-06-27,105.99,-0.116439,678604,0
59594,300724,2022-06-28,94.34,0.025018,679360,1
59595,300724,2022-06-29,96.73,-0.099143,680116,2
59596,300724,2022-06-30,87.60,0.019780,680872,3


In [18]:
#### 基于PEARS使用移动时间加权
engine = DimensionCorrcoef(features=indicator_features, thresh=0.85, method=FCorrType.F_CS_CORR)
engine.run(factors_data=indicator_data,similar_type=ECoreCorrType.E_CORE_TYPE_ROLLING, window=20)

2022-11-20 09:05:48,122 - [corrcoef.py:72] - ultron - INFO - preClosePrice,openPrice corr:0.9612728878722712


rolling_corr cost 0.719s


2022-11-20 09:05:48,882 - [corrcoef.py:72] - ultron - INFO - preClosePrice,highestPrice corr:0.8650681568161754


rolling_corr cost 0.756s


2022-11-20 09:05:49,558 - [corrcoef.py:72] - ultron - INFO - preClosePrice,lowestPrice corr:0.8862247404811089


...

2022-11-20 09:06:04,177 - [corrcoef.py:72] - ultron - INFO - atr21,p_change corr:-0.03981731568995066


rolling_corr cost 0.615s


2022-11-20 09:06:05,146 - [corrcoef.py:72] - ultron - INFO - atr21,atr14 corr:0.9695547609423967


rolling_corr cost 0.964s


,code,trade_date,preClosePrice,closePrice,turnoverVol,p_change,atr21,key,date_week
0,300002,2018-10-15,3.07,3.00,9880442,-0.023065,0.140328,328,0
1,300002,2018-10-16,3.00,2.93,10977328,-0.023610,0.139836,1079,1
2,300002,2018-10-17,2.93,3.03,12858515,0.033560,0.138891,1830,2
3,300002,2018-10-18,3.03,3.08,18707797,0.016367,0.141325,2581,3
4,300002,2018-10-19,3.08,3.19,18550310,0.035091,0.143643,3332,4
...,...,...,...,...,...,...,...,...,...
59593,300724,2022-06-27,105.99,94.34,27152642,-0.116439,5.725449,678604,0
59594,300724,2022-06-28,94.34,96.73,14268590,0.025018,5.689475,679360,1
59595,300724,2022-06-29,96.73,87.60,22555360,-0.099143,5.893310,680116,2
59596,300724,2022-06-30,87.60,89.35,13768856,0.019780,5.816961,680872,3


##### 时序降维

In [19]:
#### 皮尔逊
engine = DimensionCorrcoef(features=indicator_features, thresh=0.97, method=FCorrType.F_TS_CORR)
engine.run(factors_data=indicator_data,similar_type=ECoreCorrType.E_CORE_TYPE_PEARS)

2022-11-20 09:07:15,998 - [corrcoef.py:53] - ultron - INFO - preClosePrice,openPrice corr:0.9990624687028365
2022-11-20 09:07:17,629 - [corrcoef.py:53] - ultron - INFO - preClosePrice,highestPrice corr:0.9986346295263386
2022-11-20 09:07:19,023 - [corrcoef.py:53] - ultron - INFO - preClosePrice,lowestPrice corr:0.9987628241411326
2022-11-20 09:07:20,550 - [corrcoef.py:53] - ultron - INFO - preClosePrice,closePrice corr:0.9991260421109106
2022-11-20 09:07:22,093 - [corrcoef.py:53] - ultron - INFO - preClosePrice,turnoverVol corr:-0.34294347910930617
2022-11-20 09:07:23,492 - [corrcoef.py:53] - ultron - INFO - preClosePrice,turnoverValue corr:0.3620028039082011
2022-11-20 09:07:24,888 - [corrcoef.py:53] - ultron - INFO - preClosePrice,p_change corr:0.006148402217455833
2022-11-20 09:07:26,425 - [corrcoef.py:53] - ultron - INFO - preClosePrice,atr21 corr:0.9287615289079452
2022-11-20 09:07:27,932 - [corrcoef.py:53] - ultron - INFO - preClosePrice,atr14 corr:0.921345240245508


TS CORR::55.56%

2022-11-20 09:07:29,244 - [corrcoef.py:53] - ultron - INFO - turnoverVol,turnoverValue corr:0.4657553596042342
2022-11-20 09:07:30,691 - [corrcoef.py:53] - ultron - INFO - turnoverVol,p_change corr:0.08509969085559886
2022-11-20 09:07:32,237 - [corrcoef.py:53] - ultron - INFO - turnoverVol,atr21 corr:-0.27512203417340525
2022-11-20 09:07:33,782 - [corrcoef.py:53] - ultron - INFO - turnoverVol,atr14 corr:-0.26698029224891084


TS CORR::66.67%

2022-11-20 09:07:35,280 - [corrcoef.py:53] - ultron - INFO - turnoverValue,turnoverVol corr:0.4657553596042342
2022-11-20 09:07:36,811 - [corrcoef.py:53] - ultron - INFO - turnoverValue,p_change corr:0.10723847532244155
2022-11-20 09:07:38,365 - [corrcoef.py:53] - ultron - INFO - turnoverValue,atr21 corr:0.40882748387014844
2022-11-20 09:07:39,873 - [corrcoef.py:53] - ultron - INFO - turnoverValue,atr14 corr:0.4159510079931914


TS CORR::77.78%

2022-11-20 09:07:41,431 - [corrcoef.py:53] - ultron - INFO - p_change,turnoverVol corr:0.08509969085559886
2022-11-20 09:07:42,971 - [corrcoef.py:53] - ultron - INFO - p_change,turnoverValue corr:0.10723847532244155
2022-11-20 09:07:44,454 - [corrcoef.py:53] - ultron - INFO - p_change,atr21 corr:0.013585753034617275
2022-11-20 09:07:46,001 - [corrcoef.py:53] - ultron - INFO - p_change,atr14 corr:0.015378470943814147


TS CORR::88.89%

2022-11-20 09:07:47,444 - [corrcoef.py:53] - ultron - INFO - atr21,turnoverVol corr:-0.27512203417340525
2022-11-20 09:07:48,957 - [corrcoef.py:53] - ultron - INFO - atr21,turnoverValue corr:0.40882748387014844
2022-11-20 09:07:50,435 - [corrcoef.py:53] - ultron - INFO - atr21,p_change corr:0.013585753034617273
2022-11-20 09:07:52,008 - [corrcoef.py:53] - ultron - INFO - atr21,atr14 corr:0.9965264987197544


,code,trade_date,preClosePrice,turnoverVol,turnoverValue,p_change,atr21,key,date_week
0,300002,2018-10-15,3.07,9880442,3.021964e+07,-0.023065,0.140328,328,0
1,300002,2018-10-16,3.00,10977328,3.289364e+07,-0.023610,0.139836,1079,1
2,300002,2018-10-17,2.93,12858515,3.870353e+07,0.033560,0.138891,1830,2
3,300002,2018-10-18,3.03,18707797,5.765097e+07,0.016367,0.141325,2581,3
4,300002,2018-10-19,3.08,18550310,5.848870e+07,0.035091,0.143643,3332,4
...,...,...,...,...,...,...,...,...,...
59593,300724,2022-06-27,105.99,27152642,2.690173e+09,-0.116439,5.725449,678604,0
59594,300724,2022-06-28,94.34,14268590,1.357506e+09,0.025018,5.689475,679360,1
59595,300724,2022-06-29,96.73,22555360,2.032467e+09,-0.099143,5.893310,680116,2
59596,300724,2022-06-30,87.60,13768856,1.218505e+09,0.019780,5.816961,680872,3


In [20]:
#### 斯皮尔曼
engine = DimensionCorrcoef(features=indicator_features, thresh=0.97, method=FCorrType.F_TS_CORR)
engine.run(factors_data=indicator_data,similar_type=ECoreCorrType.E_CORE_TYPE_SPERM)

2022-11-20 09:07:58,075 - [corrcoef.py:53] - ultron - INFO - preClosePrice,openPrice corr:0.9977973511958429
2022-11-20 09:07:59,929 - [corrcoef.py:53] - ultron - INFO - preClosePrice,highestPrice corr:0.9973522196489437
2022-11-20 09:08:01,799 - [corrcoef.py:53] - ultron - INFO - preClosePrice,lowestPrice corr:0.9975128260363962
2022-11-20 09:08:03,937 - [corrcoef.py:53] - ultron - INFO - preClosePrice,closePrice corr:0.9990625214473224
2022-11-20 09:08:06,135 - [corrcoef.py:53] - ultron - INFO - preClosePrice,turnoverVol corr:-0.515476545281002
2022-11-20 09:08:08,344 - [corrcoef.py:53] - ultron - INFO - preClosePrice,turnoverValue corr:0.4499294371834481
2022-11-20 09:08:10,104 - [corrcoef.py:53] - ultron - INFO - preClosePrice,p_change corr:0.014866578792889386
2022-11-20 09:08:12,329 - [corrcoef.py:53] - ultron - INFO - preClosePrice,atr21 corr:0.9700393783927125
2022-11-20 09:08:14,117 - [corrcoef.py:53] - ultron - INFO - preClosePrice,atr14 corr:0.9689729423525376


TS CORR::55.56%

2022-11-20 09:08:16,039 - [corrcoef.py:53] - ultron - INFO - turnoverVol,turnoverValue corr:0.45559985937565123
2022-11-20 09:08:18,185 - [corrcoef.py:53] - ultron - INFO - turnoverVol,p_change corr:0.03996810792519075
2022-11-20 09:08:20,215 - [corrcoef.py:53] - ultron - INFO - turnoverVol,atr14 corr:-0.42955841312269866


TS CORR::66.67%

2022-11-20 09:08:22,224 - [corrcoef.py:53] - ultron - INFO - turnoverValue,turnoverVol corr:0.45559985937565123
2022-11-20 09:08:24,184 - [corrcoef.py:53] - ultron - INFO - turnoverValue,p_change corr:0.07860277655442409
2022-11-20 09:08:26,097 - [corrcoef.py:53] - ultron - INFO - turnoverValue,atr14 corr:0.5176126236013653


TS CORR::77.78%

2022-11-20 09:08:27,886 - [corrcoef.py:53] - ultron - INFO - p_change,turnoverVol corr:0.03996810792519076
2022-11-20 09:08:29,910 - [corrcoef.py:53] - ultron - INFO - p_change,turnoverValue corr:0.07860277655442409
2022-11-20 09:08:31,896 - [corrcoef.py:53] - ultron - INFO - p_change,atr14 corr:0.014626686022755608


,code,trade_date,preClosePrice,turnoverVol,turnoverValue,p_change,atr14,key,date_week
0,300002,2018-10-15,3.07,9880442,3.021964e+07,-0.023065,0.140576,328,0
1,300002,2018-10-16,3.00,10977328,3.289364e+07,-0.023610,0.139821,1079,1
2,300002,2018-10-17,2.93,12858515,3.870353e+07,0.033560,0.138405,1830,2
3,300002,2018-10-18,3.03,18707797,5.765097e+07,0.016367,0.142091,2581,3
4,300002,2018-10-19,3.08,18550310,5.848870e+07,0.035091,0.145513,3332,4
...,...,...,...,...,...,...,...,...,...
59593,300724,2022-06-27,105.99,27152642,2.690173e+09,-0.116439,6.322031,678604,0
59594,300724,2022-06-28,94.34,14268590,1.357506e+09,0.025018,6.225457,679360,1
59595,300724,2022-06-29,96.73,22555360,2.032467e+09,-0.099143,6.492925,680116,2
59596,300724,2022-06-30,87.60,13768856,1.218505e+09,0.019780,6.335573,680872,3


In [21]:
#### 基于PEARS使用序列＋－符号
engine = DimensionCorrcoef(features=indicator_features, thresh=0.97, method=FCorrType.F_TS_CORR)
engine.run(factors_data=indicator_data,similar_type=ECoreCorrType.E_CORE_TYPE_SIGN)

2022-11-20 09:08:41,282 - [corrcoef.py:53] - ultron - INFO - preClosePrice,openPrice corr:0.9988086848551965
2022-11-20 09:08:43,384 - [corrcoef.py:53] - ultron - INFO - preClosePrice,highestPrice corr:0.9988086848551965
2022-11-20 09:08:44,870 - [corrcoef.py:53] - ultron - INFO - preClosePrice,lowestPrice corr:0.9988086848551965
2022-11-20 09:08:46,360 - [corrcoef.py:53] - ultron - INFO - preClosePrice,closePrice corr:1.0
2022-11-20 09:08:48,013 - [corrcoef.py:53] - ultron - INFO - preClosePrice,turnoverVol corr:0.9988086848551965
2022-11-20 09:08:49,455 - [corrcoef.py:53] - ultron - INFO - preClosePrice,turnoverValue corr:0.9988086848551965
2022-11-20 09:08:51,181 - [corrcoef.py:53] - ultron - INFO - preClosePrice,p_change corr:0.49293600456391157
2022-11-20 09:08:52,686 - [corrcoef.py:53] - ultron - INFO - preClosePrice,atr21 corr:1.0
2022-11-20 09:08:54,274 - [corrcoef.py:53] - ultron - INFO - preClosePrice,atr14 corr:1.0


,code,trade_date,preClosePrice,p_change,key,date_week
0,300002,2018-10-15,3.07,-0.023065,328,0
1,300002,2018-10-16,3.00,-0.023610,1079,1
2,300002,2018-10-17,2.93,0.033560,1830,2
3,300002,2018-10-18,3.03,0.016367,2581,3
4,300002,2018-10-19,3.08,0.035091,3332,4
...,...,...,...,...,...,...
59593,300724,2022-06-27,105.99,-0.116439,678604,0
59594,300724,2022-06-28,94.34,0.025018,679360,1
59595,300724,2022-06-29,96.73,-0.099143,680116,2
59596,300724,2022-06-30,87.60,0.019780,680872,3
